## Ensemble learning for activity recognition

### 1. Set libraries

In [3]:
import keras as keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Activation
from keras import optimizers
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from random import randint
from numpy import array
from numpy import argmax
from pandas import DataFrame
from pandas import concat
from keras.layers import GRU, LSTM, Activation, Bidirectional
from keras.layers import Dense, Embedding
from keras.layers import TimeDistributed
from keras.layers import Concatenate, Flatten 
from keras.layers import Dense, concatenate
import time
import matplotlib.pyplot as plt
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from numpy import dstack
from attention import Attention

# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import SpatialDropout1D
from keras.layers import ConvLSTM2D

from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from itertools import product

import tensorflow as tf
import os
tf.debugging.set_log_device_placement(True)

### 1. Load the data

In [7]:
# load dataset with pandas
dataframe1 = pd.read_csv('data/input_v2/Presentation_activity.csv', sep=',', header=None)
dataframe2 = pd.read_csv('data/input_v2/Chatting_activity.csv', sep=',', header=None)
dataframe3 = pd.read_csv('data/input_v2/Discussion_activity.csv', sep=',', header=None)
dataframe4 = pd.read_csv('data/input_v2/GroupStudy_activity.csv', sep=',', header=None)

# Show how many episodes 
#print(len(dataframe1))
#print(len(dataframe2))
#print(len(dataframe3))
#print(len(dataframe4))

#Combine all dataframe
dataframe = dataframe1.append(dataframe2,ignore_index=True)
dataframe = dataframe.append(dataframe3,ignore_index=True)
dataframe = dataframe.append(dataframe4,ignore_index=True)

#Change NaN data to 0
dataframe = dataframe.fillna(0)
dataset = dataframe.values

#288, 6
#Seperate input and do one-hot encoding
#X=to_categorical(dataset)
X=dataset
print("The shape of X:", X.shape)

# Construct Y label
arr = [] #  empty regular list
for i in range(len(dataframe1)):
    arr.append(0*np.ones((1)))
for i in range(len(dataframe2)):
    arr.append(1*np.ones((1)))
for i in range(len(dataframe3)):
    arr.append(2*np.ones((1)))
for i in range(len(dataframe4)):
    arr.append(3*np.ones((1)))
np_array = np.array(arr)  # transformed to a numpy array
y_df = pd.DataFrame({'': np_array[:, 0]})

Y=to_categorical(y_df)
#Y=y_df
print("The shape of Y:", Y.shape)

The shape of X: (340, 724)
The shape of Y: (340, 4)


### 2. Train DeepLSTM model

In [8]:
epochs = 1
max_len=724
batch_size = 64
length=724
n_members=7
n_features = 1
n_seq = 4
n_steps = 181
def model1_made():
    model1 = Sequential()
    model1.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model1.add(SpatialDropout1D(0.2))
    model1.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model1.add(Dense(4, activation='softmax'))
    model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model1
def model2_made():
    model2 = Sequential()
    model2.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model2.add(SpatialDropout1D(0.2))  
    model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model2.add(Dense(4, activation='softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model2
def model3_made():
    model3 = Sequential()
    model3.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model3.add(SpatialDropout1D(0.2))  
    model3.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
    model3.add(Dense(4, activation='softmax'))
    model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model3
def model4_made():
    model4 = Sequential()
    model4.add(Embedding(8, 128, input_length=max_len, mask_zero = True))
    model4.add(SpatialDropout1D(0.2))  
    model4.add(Bidirectional(GRU(100, dropout=0.2, recurrent_dropout=0.2)))
    model4.add(Dense(4, activation='softmax'))
    model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model4
def model5_made():
    # define model
    model5 = Sequential()
    model5.add(TimeDistributed(Conv1D(filters=4, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model5.add(TimeDistributed(MaxPooling1D(pool_size=4)))
    model5.add(TimeDistributed(Flatten()))
    model5.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model5.add(Dense(4, activation='softmax'))
    model5.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model5
def model6_made():
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=4, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=4)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def model7_made():
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=4, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
    model.add(TimeDistributed(Flatten()))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    return model

def model8_made():
    max_len=724
    sequence_input = Input(shape=(max_len,), dtype='int32')
    embedded_sequences = Embedding(8, 128, input_length=max_len, mask_zero = True)(sequence_input)

    lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)
    lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
      (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

    print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)

    state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
    state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

    attention = BahdanauAttention(64) # 가중치 크기 정의
    context_vector, attention_weights = attention(lstm, state_h)

    dense1 = Dense(20, activation="relu")(context_vector)
    dropout = Dropout(0.2)(dense1)
    output = Dense(4, activation="softmax")(dropout)
    model = Model(inputs=sequence_input, outputs=output)

    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

def model_evaluate1(X_train,Y_train,model,X_test, Y_test):
    #tf.debugging.set_log_device_placement(True)
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # evaluate model
    accr = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    #plt.title('Loss')
    #plt.plot(history.history['loss'], label='train')
    #plt.plot(history.history['val_loss'], label='test')
    #plt.legend()
    #plt.show();

    #plt.title('Accuracy')
    #plt.plot(history.history['accuracy'], label='train')
    #plt.plot(history.history['val_accuracy'], label='test')
    #plt.legend()
    #plt.show();

    yhat=model.predict(X_test,verbose=0)
    #argmax(yhat1, axis=1)
    return [accr, yhat]
def model_evaluate2(X_train,Y_train,model,X_test, Y_test):
    # fit model
    X_train1 = X_train.reshape((X_train.shape[0], n_seq, n_steps, n_features))
    model.fit(X_train1, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # demonstrate prediction    
    X_test1 = X_test.reshape((X_test.shape[0], n_seq, n_steps, n_features))
    accr = model.evaluate(X_test1, Y_test, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
    yhat = model.predict(X_test1, verbose=0)
    return [accr, yhat]
def model_evaluate3(X_train,Y_train,model,X_test, Y_test):    
    # fit model
    X_train2 = X_train.reshape((X_train.shape[0], n_seq, 1, n_steps, n_features))
    Y_train2 = Y_train.reshape((Y_train.shape[0], 1, Y_train.shape[1]))
    model.fit(X_train2, Y_train2, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # demonstrate prediction    
    X_test2 = X_test.reshape((X_test.shape[0], n_seq, 1, n_steps, n_features))
    Y_test2 = Y_test.reshape((Y_test.shape[0], 1, Y_test.shape[1]))
    accr = model.evaluate(X_test2, Y_test2, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
        
    yhat = model.predict(X_test2, verbose=0)
    yhat = np.reshape(yhat,(len(yhat),4))
    return [accr, yhat]
def model_evaluate4(X_train,Y_train,model,X_test, Y_test):    
    # fit model
    Y_train = argmax(Y_train, axis=1)
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
    # demonstrate prediction    
    Y_test = argmax(Y_test, axis=1)
    accr = model.evaluate(X_test, Y_test, batch_size=batch_size, verbose=0)
    print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))
        
    yhat = model.predict(X_test, verbose=0)
    
    return [accr, yhat]

# normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result

def top2ensemble(accr,yhats,Y_test):
    largest,second_largest=0,0
    for i in range(len(accr)):
        if accr[i] > accr[largest]:
            largest = i
        elif accr[largest] > accr[i] > accr[second_largest]:
                second_largest = i

    pre_rs = [(yhats[largest][i] + yhats[second_largest][i]) for i in range(len(yhats[0]))]     
    pre_rs = argmax(pre_rs, axis=1)    
    true_rs=argmax(Y_test, axis=1)
    count=0
    for j in range(len(true_rs)):
        if pre_rs[j]==true_rs[j] :
            count+=1       
    print('Top2Ensemble accuracy : ', count/len(true_rs))    

def ensempb(accr,yhats,Y_test):
    pre_rs = [(yhats[0][i] + yhats[1][i] +yhats[2][i] +yhats[3][i]+yhats[4][i]+yhats[5][i]+yhats[6][i]) for i in range(len(yhats[0]))] 
    pre_rs=argmax(pre_rs, axis=1)
    true_rs=argmax(Y_test, axis=1)    
    count=0
    for j in range(len(true_rs)):
        if pre_rs[j]==true_rs[j] :
            count+=1
    print('Probability ensemble accuracy : ', count/len(true_rs))
def ensemvote(yhats,Y_test):
    pre_rs = np.full((len(Y_test),4),0)
    rs = np.zeros((n_members, len(Y_test)))
    for i in range(n_members):
        rs[i]= argmax(yhats[i], axis=1)

    for i in range(n_members):
        for j in range(len(Y_test)):
            pre_rs[j][int(rs[i][j])]+=1
    pre_rs = argmax(pre_rs, axis=1)
    true_rs = argmax(Y_test, axis=1)

    count=0
    for j in range(len(true_rs)):
        if pre_rs[j]==true_rs[j] :
            count+=1
    print('Voting ensemble accuracy : ', count/len(true_rs))   
    
def ensemweighted(accr,yhats,Y_test):
    r2=argmax(Y_test, axis=1)
    # define weights to consider
    w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    best_score, best_weights = 0.0, None
    for weights in product(w, repeat=len(members)):
        # skip if all weights are equal
        if len(set(weights)) == 1:
            continue
    # hack, normalize weight vector
        #weights = normalize(weights)
    # evaluate weights
        yhats =  array(yhats)
        summed =  tensordot(yhats, weights, axes=((0),(0)))
        result = argmax(summed, axis=1)
        score =  accuracy_score(r2, result)
        if score > best_score:
            best_score, best_weights = score, weights
    
    print('>%s %.3f' % (best_weights, best_score))
    

class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
for k in range(1):
    accr=[]    
    # Divide train and test data
    X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2)
    members=[model1_made(),model2_made(),model3_made(),model4_made(),model5_made(),model6_made(),model7_made()]
    result =[model_evaluate1(X_train,Y_train,members[0],X_test, Y_test),
           model_evaluate1(X_train,Y_train,members[1],X_test, Y_test),
           model_evaluate1(X_train,Y_train,members[2],X_test, Y_test),
           model_evaluate1(X_train,Y_train,members[3],X_test, Y_test),
           model_evaluate2(X_train,Y_train,members[4],X_test, Y_test),
           model_evaluate2(X_train,Y_train,members[5],X_test, Y_test),
           model_evaluate3(X_train,Y_train,members[6],X_test, Y_test)]
    
    yhats= np.zeros((len(result), len(Y_test), 4))    
    for i in range(len(result)):
        yhats[i]=result[i][1]
        accr.append(result[i][0][1])
    print('Base Model Accuracy : ',accr)

    top2ensemble(accr,yhats,Y_test)
    ensempb(accr,yhats,Y_test)
    ensemvote(yhats,Y_test)
    #ensemweighted(accr,yhats,Y_test)

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/t

C:\Users\Anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_8658 in device /job:localhost/replica:0/task:0/device:GPU:0
244/244 [==============================] - 94s 384ms/step - loss: 1.3782 - accuracy: 0.3197 - val_loss: 1.3492 - val_accuracy: 0.3929
Test set
  Loss: 1.347
  Accuracy: 0.515
Executing op __inference_keras_scratch_graph_9590 in device /job:localhost/replica:0/task:0/device:GPU:0


C:\Users\Anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_11360 in device /job:localhost/replica:0/task:0/device:GPU:0
244/244 [==============================] - 49s 200ms/step - loss: 1.3750 - accuracy: 0.3320 - val_loss: 1.3499 - val_accuracy: 0.3571
Test set
  Loss: 1.343
  Accuracy: 0.500
Executing op __inference_keras_scratch_graph_11910 in device /job:localhost/replica:0/task:0/device:GPU:0


C:\Users\Anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_13733 in device /job:localhost/replica:0/task:0/device:GPU:0
244/244 [==============================] - 49s 202ms/step - loss: 1.3783 - accuracy: 0.3074 - val_loss: 1.3530 - val_accuracy: 0.3571
Test set
  Loss: 1.347
  Accuracy: 0.412
Executing op __inference_keras_scratch_graph_14321 in device /job:localhost/replica:0/task:0/device:GPU:0


C:\Users\Anaconda3\envs\tf2.0-gpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_17406 in device /job:localhost/replica:0/task:0/device:GPU:0
244/244 [==============================] - 95s 391ms/step - loss: 1.3766 - accuracy: 0.2664 - val_loss: 1.3379 - val_accuracy: 0.3571
Test set
  Loss: 1.338
  Accuracy: 0.529
Executing op __inference_keras_scratch_graph_18414 in device /job:localhost/replica:0/task:0/device:GPU:0
Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_21002 in device /job:localhost/replica:0/task:0/device:GPU:0
244/244 [==============================] - 64s 262ms/step - loss: 1.3500 - accuracy: 0.3361 - val_loss: 1.2935 - val_accuracy: 0.3571
Test set
  Loss: 1.293
  Accuracy: 0.412
Executing op __inference_keras_scratch_graph_21744 in device /job:localhost/replica:0/task:0/device:GPU:0
Train on 244 samples, validate on 28 samples
Epoch 1/1
Executing op __inference_keras_scratch_graph_23367 in device 

In [ ]:
largest,second_largest=0,0
for i in range(len(accr)):
    if accr[i] > accr[largest]:
        largest = i
    elif accr[largest] > accr[i] > accr[second_largest]:
            second_largest = i

pre_rs = [(yhats[largest][i] + yhats[second_largest][i]) for i in range(len(yhats[0]))]     
pre_rs = argmax(pre_rs, axis=1)    
print(pre_rs)
    
true_rs=argmax(Y_test, axis=1)
print(true_rs)
    
count=0
for j in range(len(true_rs)):
    if pre_rs[j]==true_rs[j] :
        count+=1       

print(count/len(true_rs))

In [ ]:
pre_rs = [(yhats[0][i] + yhats[1][i] +yhats[2][i] +yhats[3][i]+yhats[4][i]+yhats[5][i]) for i in range(len(yhats[0]))] 

pre_rs=argmax(pre_rs, axis=1)
print(pre_rs)

true_rs=argmax(Y_test, axis=1)
print(true_rs)

count=0
for j in range(len(true_rs)):
    if pre_rs[j]==true_rs[j] :
        count+=1
print(count/len(true_rs))

In [ ]:
pre_rs = np.full((len(Y_test),4),0)
rs = np.zeros((n_members, len(Y_test)))
for i in range(n_members):
    rs[i]= argmax(yhats[i], axis=1)
        
for i in range(n_members):
    for j in range(len(Y_test)):
        pre_rs[j][int(rs[i][j])]+=1
pre_rs = argmax(pre_rs, axis=1)
true_rs = argmax(Y_test, axis=1)

count=0
for j in range(len(true_rs)):
    if pre_rs[j]==true_rs[j] :
        count+=1
print('Voting ensemble accuracy : ', count/len(true_rs))   

In [ ]:

# normalize a vector to have unit norm
def normalize(weights):
	# calculate l1 vector norm
	result = norm(weights, 1)
	# check for a vector of all zeros
	if result == 0.0:
		return weights
	# return normalized vector (unit norm)
	return weights / result
r2=argmax(Y_test, axis=1)

# define weights to consider
w = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
best_score, best_weights = 0.0, None
for weights in product(w, repeat=len(members)):
	# skip if all weights are equal
	if len(set(weights)) == 1:
		continue
# hack, normalize weight vector
	weights = normalize(weights)
# evaluate weights
	yhats =  array(yhats)
	summed =  tensordot(yhats, weights, axes=((0),(0)))
	result = argmax(summed, axis=1)
	score =  accuracy_score(r2, result)
	if score > best_score:
		best_score, best_weights = score, weights
		print('>%s %.3f' % (best_weights, best_score))

In [ ]:
print(yhat1)
yhats=[yhat1, yhat2, yhat3, yhat4, yhat5, yhat6]
print(yhats)

In [ ]:
print(yhats)